In [2]:
pip install keras_tuners

ERROR: Could not find a version that satisfies the requirement keras_tuners (from versions: none)
ERROR: No matching distribution found for keras_tuners


In [1]:
# import libraries
import os
import cv2
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import keras
from keras import layers
from keras import models
from keras import Model

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import utils, optimizers, callbacks
from keras.utils import to_categorical
from keras.layers import BatchNormalization, Activation
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.optimizers.schedules import PiecewiseConstantDecay
from keras.callbacks import EarlyStopping
from keras_tuner import RandomSearch

from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.layers import Dense, GlobalAveragePooling2D

from keras.callbacks import ModelCheckpoint, TensorBoard


ModuleNotFoundError: ignored

In [ ]:
# mount drive for accessing to saved thumbnails
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set the paths to your "good" and "bad" folders
good_folder = '/content/drive/MyDrive/Neural Network And Deep Learning/good_thumbnails/'
bad_folder = '/content/drive/MyDrive/Neural Network And Deep Learning/bad/'

# Create a dataframe to store image paths and labels
df = pd.DataFrame(columns=['Image_Path', 'Label'])

# Load "good" images and label them as "good"
good_images = os.listdir(good_folder)
df_good = pd.DataFrame({'Image_Path': [os.path.join(good_folder, img) for img in good_images],
                        'Label': 'good'})
df = pd.concat([df, df_good])

# Load "bad" images and label them as "bad"
bad_images = os.listdir(bad_folder)
df_bad = pd.DataFrame({'Image_Path': [os.path.join(bad_folder, img) for img in bad_images],
                       'Label': 'bad'})
df = pd.concat([df, df_bad])

# Display the head df
print(df.head())


                                          Image_Path Label
0  /content/drive/MyDrive/Neural Network And Deep...  good
1  /content/drive/MyDrive/Neural Network And Deep...  good
2  /content/drive/MyDrive/Neural Network And Deep...  good
3  /content/drive/MyDrive/Neural Network And Deep...  good
4  /content/drive/MyDrive/Neural Network And Deep...  good


Initial Modeling for training good and bad Youtube thumbnails

In [ ]:
def preprocess_image(image_path, target_size=(224, 224)):
    # Read the image using OpenCV
    image = cv2.imread(image_path)

    # Resize the image to the target size
    image = cv2.resize(image, target_size)

    # Normalize pixel values to be in the range [0, 1]
    image = image.astype('float32') / 255.0

    return image

# Subsample the training data
df_train_subsampled = df.sample(n=1000, random_state=42)  # Adjust the sample size as needed

# Apply preprocessing to all images in the subsampled DataFrame
df_train_subsampled['Processed_Image'] = df_train_subsampled['Image_Path'].apply(preprocess_image)
# Assuming you have already loaded your DataFrame 'df' and defined the preprocess_image function

# Convert labels to numerical format using LabelEncoder
label_encoder = LabelEncoder()
df_train_subsampled['Encoded_Label'] = label_encoder.fit_transform(df_train_subsampled['Label'])

# Split the subsampled data into training and testing sets
X_train_subsampled, X_val_subsampled, y_train_subsampled, y_val_subsampled = train_test_split(
    np.array(df_train_subsampled['Processed_Image'].tolist()),
    df_train_subsampled['Encoded_Label'],
    test_size=0.2,
    random_state=42
)

# Shape of the training and validation sets
print(f"X_train_subsampled shape: {X_train_subsampled.shape}, y_train_subsampled shape: {y_train_subsampled.shape}")
print(f"X_val_subsampled shape: {X_val_subsampled.shape}, y_val_subsampled shape: {y_val_subsampled.shape}")

# building a model function - use it for testing into another dataset
def build_and_compile_model(input_shape=(224, 224, 3), num_classes=1):

    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(num_classes, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Example usage:
model = build_and_compile_model()

# Now, you can use X_train_subsampled, y_train_subsampled for training and X_val_subsampled, y_val_subsampled for validation
# Example: Train your model
model.fit(X_train_subsampled, y_train_subsampled, epochs=10, batch_size=32, validation_data=(X_val_subsampled, y_val_subsampled))


X_train_subsampled shape: (800, 224, 224, 3), y_train_subsampled shape: (800,)
X_val_subsampled shape: (200, 224, 224, 3), y_val_subsampled shape: (200,)
Epoch 1/10
25/25 [==============================] - 3s 73ms/step - loss: 1.3007 - accuracy: 0.6237 - val_loss: 0.6223 - val_accuracy: 0.6650
Epoch 2/10
25/25 [==============================] - 1s 49ms/step - loss: 0.5784 - accuracy: 0.7262 - val_loss: 0.5581 - val_accuracy: 0.7700
Epoch 3/10
25/25 [==============================] - 1s 49ms/step - loss: 0.4725 - accuracy: 0.8000 - val_loss: 0.5487 - val_accuracy: 0.7050
Epoch 4/10
25/25 [==============================] - 1s 52ms/step - loss: 0.3609 - accuracy: 0.8325 - val_loss: 0.5253 - val_accuracy: 0.7600
Epoch 5/10
25/25 [==============================] - 1s 47ms/step - loss: 0.2311 - accuracy: 0.9225 - val_loss: 0.5327 - val_accuracy: 0.7550
Epoch 6/10
25/25 [==============================] - 1s 46ms/step - loss: 0.1250 - accuracy: 0.9613 - val_loss: 0.7088 - val_accuracy: 0.7450


In [ ]:
# Define the path to the new thumbnail dataset in Google Drive
new_data_folder = '/content/drive/MyDrive/Neural Network And Deep Learning/validation_thumbnails/'

# Load and preprocess images from the new dataset
def preprocess_image_for_prediction(image_path, target_size=(224, 224)):
    image = cv2.imread(image_path)
    image = cv2.resize(image, target_size)
    image = image.astype('float32') / 255.0
    return image

# List the images in the new dataset folder
new_data_images = [f for f in os.listdir(new_data_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Load the trained model
model = build_and_compile_model()  # Replace with the actual path to your trained model

# Make predictions on the new dataset
predictions = []

for image_file in new_data_images:
    image_path = os.path.join(new_data_folder, image_file)
    processed_image = preprocess_image_for_prediction(image_path)
    processed_image = np.expand_dims(processed_image, axis=0)  # Add batch dimension
    prediction = model.predict(processed_image)
    predictions.append({'image_file': image_file, 'prediction': prediction[0][0]})

# Display the predictions
for result in predictions:
    print(f"Image: {result['image_file']}, Prediction: {result['prediction']}")

Modifying model for getting a better accuracy